In [2]:
import numpy as np

def create_random_array(w, h):
    array = np.random.randint(0, 9, size=(h, w))
    return array

def create_indexed_2d_array(shape):
    flat_array = [i+1 for i in range(shape * shape)]
    return np.array([flat_array[i*shape:(i+1)*shape] for i in range(shape)])

print(create_indexed_2d_array(5))

size = 3
image = np.array([[1, 2, 3],[4, 5, 6],
        [7, 8, 9]])
print(image)

kernel = [
    [1, 2],
    [3, 4]
]

def flatten_array(array):
    return array.flatten(order='C')

flattened_image = flatten_array(image)

[[ 1  2  3  4  5]
 [ 6  7  8  9 10]
 [11 12 13 14 15]
 [16 17 18 19 20]
 [21 22 23 24 25]]
[[1 2 3]
 [4 5 6]
 [7 8 9]]


In [9]:
def convolve2d(input_matrix, kernel):
    # Dimensions of the input and kernel
    input_rows = len(input_matrix)
    input_cols = len(input_matrix[0])
    kernel_size = len(kernel)
    
    # Define the output matrix dimensions (for valid convolution)
    output_rows = input_rows - kernel_size + 1
    output_cols = input_cols - kernel_size + 1
    
    # Initialize output matrix with zeros
    output = [[0 for _ in range(output_cols)] for _ in range(output_rows)]
    
    # Perform convolution
    for i in range(output_rows):
        for j in range(output_cols):
            # Initialize the sum for this position
            sum_value = 0
            for ki in range(kernel_size):
                for kj in range(kernel_size):
                    # Multiply input element with the corresponding kernel element
                    sum_value += input_matrix[i + ki][j + kj] * kernel[ki][kj]
            # Assign the sum to the output matrix
            output[i][j] = sum_value
    
    return output

print(convolve2d(image, kernel))

[[37, 47], [67, 77]]


In [10]:
import numpy as np

def kernel2toeplitz(K, input_shape):
    m, n = input_shape
    k, _ = K.shape  # assuming kernel is square for simplicity
    
    output_rows = m - k + 1
    output_cols = n - k + 1
    
    toeplitz_cols = m * n  # The flattened input size
    toeplitz_rows = output_rows * output_cols  # Number of output pixels
    
    toeplitz_matrix = np.zeros((toeplitz_rows, toeplitz_cols))
    
    row = 0
    for i in range(output_rows):
        for j in range(output_cols):
            # Create a flattened mask with the kernel shifted at (i, j)
            flattened_kernel = np.zeros((m, n))
            flattened_kernel[i:i+k, j:j+k] = K  # Place the kernel at (i, j)
            toeplitz_matrix[row, :] = flattened_kernel.flatten()  # Flatten it into a row
            row += 1
    
    return toeplitz_matrix

def convolve_using_toeplitz(input_matrix, K):
    input_shape = input_matrix.shape
    toeplitz_matrix = kernel2toeplitz(K, input_shape)
    
    # Flatten the input matrix
    flattened_input = input_matrix.flatten()
    
    # Perform matrix multiplication (Toeplitz matrix with flattened input)
    conv_result = toeplitz_matrix @ flattened_input
    
    # Reshape the result to the proper output shape
    output_shape = (input_shape[0] - K.shape[0] + 1, input_shape[1] - K.shape[1] + 1)
    conv_result = conv_result.reshape(output_shape)
    
    return conv_result

# Example usage
input_matrix = np.array([[1, 2, 3],
                         [4, 5, 6],
                         [7, 8, 9]])

K = np.array([[1, 2],
              [3, 4]])  # Example kernel
toeplitz_k = kernel2toeplitz(K, (3,3))

conv_result = convolve_using_toeplitz(input_matrix, K)
print("Convolution Result via Toeplitz Matrix Multiplication:\n", conv_result)


Convolution Result via Toeplitz Matrix Multiplication:
 [[37. 47.]
 [67. 77.]]


In [4]:
import numpy as np

def save_as_mem_row_major(array, filename):
    """
    Save the 2D numpy array to a .mem file in row-major order.
    
    Parameters:
    - array: 2D numpy array
    - filename: The name of the .mem file to save the array into
    """
    with open(filename, 'w') as f:
        for row in array:
            for value in row:
                # Convert to hexadecimal (or you can use a different format if needed)
                f.write(f"{value:02X}\n")  # Assuming you want to save it as 2-digit hex
    print(f"Saved in row-major order to {filename}")

def save_as_mem_column_major(array, filename):
    """
    Save the 2D numpy array to a .mem file in column-major order.
    
    Parameters:
    - array: 2D numpy array
    - filename: The name of the .mem file to save the array into
    """
    with open(filename, 'w') as f:
        for col in range(array.shape[1]):  # Iterate over columns first (column-major order)
            for row in range(array.shape[0]):
                value = int(array[row, col])
                # Convert to hexadecimal (or you can use a different format if needed)
                f.write(f"{value:02X}\n")  # Assuming you want to save it as 2-digit hex
    print(f"Saved in column-major order to {filename}")

# Save in row-major order
save_as_mem_row_major(image, 'ifmap.mem')

test = create_random_array(5,5)

print(test)
# Save in column-major order
save_as_mem_row_major(test, 'weights.mem')

[
    [4, 0, 2, 0, 1, 5, 1, 2, 7],
    [0, 2, 0, 1, 5, 7, 2, 7, 7],
    [2, 0, 2, 5, 7, 6, 7, 7, 1],
    [0, 1, 5, 1, 2, 7, 1, 2, 4],
    [1, 5, 7, 2, 7, 7, 2, 4, 2],
    [5, 7, 6, 7, 7, 1, 4, 2, 0],
    [1, 2, 7, 1, 2, 4, 1, 6, 6],
    [2, 7, 7, 2, 4, 2, 6, 6, 6],
    [7, 7, 1, 4, 2, 0, 6, 6, 2]
]

Saved in row-major order to ifmap.mem
[[3 6 4 5 0]
 [6 0 5 8 1]
 [2 6 1 2 6]
 [2 5 8 2 8]
 [0 0 7 1 7]]
Saved in row-major order to weights.mem


[[4, 0, 2, 0, 1, 5, 1, 2, 7],
 [0, 2, 0, 1, 5, 7, 2, 7, 7],
 [2, 0, 2, 5, 7, 6, 7, 7, 1],
 [0, 1, 5, 1, 2, 7, 1, 2, 4],
 [1, 5, 7, 2, 7, 7, 2, 4, 2],
 [5, 7, 6, 7, 7, 1, 4, 2, 0],
 [1, 2, 7, 1, 2, 4, 1, 6, 6],
 [2, 7, 7, 2, 4, 2, 6, 6, 6],
 [7, 7, 1, 4, 2, 0, 6, 6, 2]]

In [12]:
import numpy as np

def save_as_mem_concatenated_row_values(array, filename):
    """
    Save the 2D numpy array to a .mem file where the values in each row are concatenated
    without spaces, and each row is written on a new line.
    
    Parameters:
    - array: 2D numpy array
    - filename: The name of the .mem file to save the array into
    """
    with open(filename, 'w') as f:
        for row in array:
            # Concatenate all values in the row into a single string without spaces
            row_str = ''.join(f"{int(value):02X}" for value in row)  # Two-digit hexadecimal
            f.write(f"{row_str}\n")  # Write the concatenated row string and add a newline
    print(f"Saved array with concatenated row values to {filename}")

# Example usage
array = np.array([[1, 2, 3],
                  [4, 5, 6],
                  [7, 8, 9]])



# Save the array into a .mem file
save_as_mem_concatenated_row_values(create_random_array(5,5), 'weights.mem')


Saved array with concatenated row values to weights.mem


In [13]:
print(create_random_array(5,5))

[[8 7 0 2 7]
 [0 4 7 3 5]
 [6 2 5 8 8]
 [8 3 5 5 0]
 [4 7 7 8 7]]


In [19]:
print("070607050100020004" [::-1])

400020001050706070


In [5]:
# Prog Rep 7 code
router_mem = create_indexed_2d_array(9)
save_as_mem_row_major(router_mem, "router_9x9.mem")

Saved in row-major order to router_9x9.mem
